In [1]:
# Importing Libraries
import pandas as pd
import glob ##For reading multiple files
import numpy as np
import dask.dataframe as dd


In [2]:
folder_path = 'NY-citibike-tripdata/*.csv' ##Path to the folder containing the data files
csv_files = glob.glob(folder_path) ##Reading all the files in the folder
csv_files = csv_files[:1] ##Reading only the first 5 files
data_df = pd.concat(
    [pd.read_csv(file, dtype={5: str, 7: str}) for file in csv_files]
)  ##Concatenating all the files to form a


###Data Exploration & Pre-Processing


In [3]:
data_df.columns = data_df.columns.str.upper() ##Converting column names to uppercase


In [4]:

data_df.isnull().sum() ##Checking for missing values
data_df = data_df.dropna() ##Dropping missing values
data_df = data_df.drop_duplicates() ##Dropping duplicates


In [5]:
##Renaming columns
data_df = data_df.rename(columns={'RIDEABLE_TYPE': 'BIKE_TYPE', 'START_STATION_NAME': 'START_STATION',
                                  'END_STATION_NAME': 'END_STATION','STARTED_AT':'START_DT','ENDED_AT':'END_DT'}) 
data_df['END_DT'] = pd.to_datetime(data_df['END_DT'])
data_df['START_DT'] = pd.to_datetime(data_df['START_DT'])

In [6]:
# Created function to calculate distance using the Haversine formula
def haversine(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(np.radians, [lat1, lon1, lat2, lon2])
    
    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = np.sin(dlat / 2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon / 2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
    r = 3958.8  # Radius of Earth in miles
    return r * c

# Apply the function to each row in the DataFrame
data_df['DISTANCE_MILES'] = data_df.apply(
    lambda row: haversine(row['START_LAT'], row['START_LNG'],
                          row['END_LAT'], row['START_LNG']), axis=1
)
data_df['DISTANCE_MILES'] = data_df['DISTANCE_MILES'].round(2) ##Rounding distance to 2 decimal places

In [7]:
# Calculating trips total duration in  mins & seconds
total_seconds = (data_df['END_DT'] - data_df['START_DT']).dt.total_seconds()
data_df['DURATION_MM_SS'] = (total_seconds // 60).astype(int).astype(str).str.zfill(2) + ':' + \
                            (total_seconds % 60).astype(int).astype(str).str.zfill(2)
                            
# Finding AVG MPH
duration_hours = total_seconds / 3600
data_df['AVG_MPH'] = data_df['DISTANCE_MILES'] / duration_hours
data_df

,RIDE_ID,BIKE_TYPE,START_DT,END_DT,START_STATION,START_STATION_ID,END_STATION,END_STATION_ID,START_LAT,START_LNG,END_LAT,END_LNG,MEMBER_CASUAL,DISTANCE_MILES,DURATION_MM_SS,AVG_MPH
0,DC1CB984821DFFF7,classic_bike,2023-01-07 15:36:53.430,2023-01-07 15:39:45.406,Vesey St & Church St,5216.06,Albany St & Greenwich St,5145.02,40.712220,-74.010472,40.709267,-74.013247,member,0.20,02:51,4.186631
1,C00CA02971557F16,classic_bike,2023-01-04 19:23:01.234,2023-01-04 19:34:57.151,Lispenard St & Broadway,5391.06,St Marks Pl & 1 Ave,5626.13,40.719392,-74.002472,40.727791,-73.985649,member,0.58,11:55,2.916539
2,C753AE5EBD8458F9,classic_bike,2023-01-20 09:22:19.894,2023-01-20 10:23:24.255,3 Ave & Schermerhorn St,4437.01,State St & Smith St,4522.07,40.686832,-73.979677,40.689170,-73.988600,member,0.16,61:04,0.157190
3,E4415A543C1972A7,classic_bike,2023-01-24 10:38:01.135,2023-01-24 10:41:40.749,E 5 St & Ave A,5626.06,E 1 St & 1 Ave,5593.01,40.724790,-73.984301,40.723356,-73.988650,member,0.10,03:39,1.639240
4,BD52A87B215877C7,electric_bike,2023-01-13 10:17:38.192,2023-01-13 10:33:59.099,W 54 St & 11 Ave,6955.05,Washington St & Gansevoort St,6039.06,40.768292,-73.992563,40.739323,-74.008119,member,2.00,16:20,7.340145
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999995,58FE57A95A48C3B9,electric_bike,2023-01-16 12:56:52.115,2023-01-16 13:06:51.277,5 Ave & E 87 St,7323.09,Amsterdam Ave & W 66 St,7149.05,40.782804,-73.959659,40.774667,-73.984706,member,0.56,09:59,3.364699
999996,D1E531A3C0AE3355,classic_bike,2023-01-20 06:21:24.809,2023-01-20 06:46:56.658,7 Ave & 41 St,3192.05,Duffield St & Willoughby St,4596.05,40.647379,-74.000911,40.692216,-73.984284,member,3.10,25:31,7.285313
999997,3DD5E1615B32FD6F,classic_bike,2023-01-25 06:21:19.057,2023-01-25 06:49:00.240,7 Ave & 41 St,3192.05,Duffield St & Willoughby St,4596.05,40.647379,-74.000911,40.692216,-73.984284,member,3.10,27:41,6.718104
999998,1AA479833FE782AE,classic_bike,2023-01-11 07:49:02.116,2023-01-11 07:58:38.117,Sterling Pl & Bedford Ave,3993.03,Plaza St West & Flatbush Ave,4010.13,40.672695,-73.954131,40.675021,-73.971115,member,0.16,09:36,0.999998


In [8]:
data_df['DAY_OF_WEEK'] = data_df['START_DT'].dt.day_name() ##Extracting day of the week 

In [9]:
#Creating a function to get the time of the day
def time_of_day(hour):
    if 5 <= hour < 12:
        return 'Morning'
    elif 12 <= hour < 17:
        return 'Afternoon'
    elif 17 <= hour < 21:
        return 'Evening'
    else:
        return 'Night'
data_df['TIME_OF_DAY'] = data_df['START_DT'].dt.hour.apply(time_of_day) ##Applying the function to get the time of the day

In [10]:
data_df['START_DT'] = data_df['START_DT'].dt.strftime('%Y-%m-%d') ##Converting start date to string
data_df['END_DT'] = data_df['END_DT'].dt.strftime('%Y-%m-%d') ##Converting end date to string
data_df = data_df.drop(
    ["START_STATION_ID", "END_STATION_ID", "END_LAT", "END_LNG"], axis=1
)  ##Dropping columns
data_df.shape

(995687, 14)

In [19]:
data_df.head(500).to_csv('citibike_data.csv', index=False) ##Saving the data to a csv file